In [ ]:
'''
This notebook performs analysis on datasets with zip codes using Pandas, bokeh, and scipy (kmeans2)
The data I used was about 2800 rows; the format was price data in the format "ID, zip, Price" as the column headers

'''
import pandas as pd
import numpy as np
from bokeh.charts import Bar, output_notebook, show

In [ ]:
#load the data
path = 'data/workbook.csv'
zip_count = pd.read_csv(path)
zip_count.head()

In [ ]:
#count them up and graph them; here I only graph the top 10, using bokeh
zip_count['ZCOUNT'] = zip_count.groupby('zip')['zip'].transform('count')
zip_count = zip_count.sort(['ZCOUNT'],ascending=0)
zip_count = zip_count.drop_duplicates(subset='zip')
zip_count = zip_count.reset_index(drop=True)
top_10 = zip_count.ix[0:9,1:4]
cat = []
zips = top_10['zip']
for item in zips:
    item = str(item)
    cat.append(item.zfill(5))
TOOLS = "pan, previewsave, reset"
bar = Bar(top_10['ZCOUNT'], cat, title = 'Zip Code Frequency', tools=TOOLS, xlabel = 'Zip Code', ylabel = 'Frequency', stacked=True)
show(bar)

In [ ]:
#next we need to convert the zip codes into lat/longs for further analysis; here I use the google API
import requests
zip_count['zip'] = zip_count['zip'].astype(str)
zip_list = zip_count['zip'].tolist()
urlp1 = 'https://maps.googleapis.com/maps/api/geocode/json?address='
urlp2 = '&key=YOUR_KEY_HERE'

In [ ]:
#use results to receive the JSON from google and append it to a list
lat = []
lng = []
for item in zip_list:
    results = requests.get(urlp1+str(item)+urlp2).json()
    lat.append(float(results["results"][0]["geometry"]["location"]["lat"]))
    lng.append(float(results["results"][0]["geometry"]["location"]["lng"]))
zip_count['LAT'] = lat
zip_count['LNG'] = lng

In [ ]:
#save the geo information from google to a csv to ensure we don't loose it
zip_count.to_csv('workbook_withgeo.csv')

In [ ]:
#lets do some basic cluster analysis on the lat/longs, using scipy
from scipy.cluster.vq import kmeans2

conus = zip_count
slon = conus['LNG'].tolist()
slat = conus['LAT'].tolist()

#we extract the values so we can iterate our kmeans function, number is the number of centroids to produce.
number = 5
myk = conus[['LAT', 'LNG']].values
roid = kmeans2(myk, number, iter = 10)

#and produce centroids, which we can use later ...

centroids = roid[0]
rlon = []
rlat = []
for item in centroids:
    rlat.append(item[0])
    rlon.append(item[1])

In [ ]:
#Bokeh has a very nice vector map of the continental US - we can use it
#it requires downloading the sample data, which you can do per the instructions here: http://bokeh.pydata.org/en/latest/docs/quickstart.html#sample-data
from bokeh.sampledata import us_states,us_counties
from bokeh.plotting import figure
us_states = us_states.data.copy()
us_counties = us_counties.data.copy()

#I mainly want to look at the continetal US
del us_states["HI"]
del us_states["AK"]
state_xs = [us_states[code]["lons"] for code in us_states]
state_ys = [us_states[code]["lats"] for code in us_states]
'''
you can add counties by including this code, but it slows the notebook down
county_xs=[us_counties[code]["lons"] for code in us_counties if us_counties[code]["state"] not in ["ak", "hi", "pr", "gu", "vi", "mp", "as"]]
county_ys=[us_counties[code]["lats"] for code in us_counties if us_counties[code]["state"] not in ["ak", "hi", "pr", "gu", "vi", "mp", "as"]]
'''
ttext = "Cluster Analysis using Kmeans2 for Test Data"
TOOLS = "pan, wheel_zoom,box_zoom,reset,previewsave"
p = figure(plot_width=1100, plot_height=700, toolbar_location = 'left', title=ttext, tools=TOOLS)

#here we overlay the scatter plot of our centroids in blue with our zipcodes in red
p.scatter(slon,slat, size=4,color='red',alpha=0.7, legend='Locations')
p.patches(state_xs, state_ys, fill_alpha=0.0,line_color="#884444", line_width=1)
#p.patches(county_xs, county_ys, fill_color='white', fill_alpha=0,line_color="gray", line_width=0.5, line_alpha=0.5)
p.scatter(rlon,rlat, size=12,color='blue',alpha=0.7, legend='Centroids')
show(p)

In [ ]:
#Now, lets put them into a kml so we can view them in google earth:
import simplekml
import csv
inputfile = 'workbook_withgeo.csv' #this is the file we just created to save the lat-long data
with open(inputfile, 'r') as csvfile:
    csvreader = csv.reader(csvfile)
    next(csvreader)
    for row in csvreader:
        pnt = kml.newpoint()
        pnt.altitudemode = 'relativeToGround'
        #inspect your file to get the values I use below ...
        pnt.name = row[1]
        #we'll use the price as the height
        height = row[3]+'0000'
        pnt.description = str('\n price :'+row[3])
        #long, lat
        pnt.coords = [(row[6],row[5], height)]
        pnt.style.iconstyle.color = simplekml.Color.red
        pnt.style.iconstyle.scale = .3
        pnt.style.labelstyle.scale = .75

kml.save('test.kml')